# The Ninapro Dataset
[Can be found here](https://ninapro.hevs.ch/)

In this week we used the Ninapro dataset DB1.



In [ ]:
import os

root_dir = os.getcwd()
rel_dir = 'datasets/ninapro/db1/s1/S1_A1_E1.mat'
file_path = os.path.join(root_dir, rel_dir)

In [ ]:
import numpy as np
from scipy.io import loadmat as ld

mat_data = ld(rel_dir, appendmat=False)

## Available fields in the ninapro dataset

In [ ]:
keys = list(mat_data.keys())
keys

## Loading all the subjects' data into a 'struct'.
This struct 'data', will have the following structure:

Data is an array of n subjects
each subject containing 3 experiments
each experiment containing a dictionary (key-value pair)
we'll load the sEMG signals from all the channels, and the cyberglove data, for the hand joint angle regression task.

We also computed the angle differences between samples, to model the correlation between joint angle variation and the incoming EMG signal. This variation per sample is given by the following equation:

\begin{align*}
    \delta_i\left[n\right] &= \begin{cases}
        0 &\quad,\text{if } n \leq 1\\
        g_i\left[n+1\right]-g_i\left[n\right]
    \end{cases}
\end{align*}


Where $i$ is a cyberglove joint ($i \in \left[1,22\right], i \in \mathbb{R}$), $\delta\left[n\right]$ is the angle variation (in $\degree$) at sample $n$, $g\left[n\right]$ is the cyberglove's uncalibrated angle at sample $n$.

## Normalize data
The emg signal was normalized from each channel's relative minimum and max, to $\left[0,1\right]$
while the glove angles were normalized, from $\left[0,360\right]$ to $\left[0,1\right]$, using:


$$x' = \frac{x - \text{min}~x}{\text{max}~x - \text{min}~x}$$

And 'delta', was normalized from $\left[-360,360\right]$ to $\left[-1,1\right]$, using:

$$x'' = 2 \frac{x - \text{min}~x}{\text{max}~x - \text{min}~x}$$

(source: https://stats.stackexchange.com/questions/178626/how-to-normalize-data-between-1-and-1)

In [ ]:
import os
from scipy.io import loadmat
import numpy as np

# Define the structure array to store the data
num_subjects = 27
data = []
dataset_path = os.path.join("datasets", "ninapro", "db1")

def normalizer(data_array,min_val,max_val,type):
    
    match type:
        case "uni": 
            return (data_array - min_val) / (max_val - min_val)            
        case "bi":
            return 2 * ( (data_array - min_val) / (max_val - min_val ) ) - 1
        case __:
            raise Exception(f"{type} type normalization not yet implemented.")

# Loop through each subject
    # Initialize lists to hold the experimental data

emg_data = []
delta_data = []

for subj_idx in range(1, num_subjects + 1):
    # Set up the subject directory name and path
    subject_dir = f"s{subj_idx}"
    subject_path = os.path.join(dataset_path, subject_dir)


    # Loop through each experiment for the current subject
    for exp_idx in range(1, 4):
        # Set up the experiment file name and path
        exp_name = f"S{subj_idx}_A1_E{exp_idx}.mat"
        exp_path = os.path.join(subject_path, exp_name)

        # Load the MATLAB file
        mat_file = loadmat(exp_path)

        # Extract the relevant information from the loaded dictionary
        emg = mat_file['emg']
        glove = mat_file['glove']
        
        # glove delta
        delta = np.diff(glove, axis=0) # compute differences
        delta = np.pad(delta, ((1, 0), (0, 0)), mode='constant') # first sample delta = 0

        # normalize data
        norm_emg = normalizer(emg, np.min(emg, axis = 0), np.max(emg, axis = 0) , "uni") # min-max channel, [0;1]
        #norm_glove = normalizer(glove,0,360, "uni") # min-max channel, [0;1]
        norm_delta = normalizer(delta,-360,360, "bi")

        emg_data.append(norm_emg)
        delta_data.append(norm_delta)
        
        # Append the list of experiments to the data list
emg_data = np.array(emg_data, dtype=object)
delta_data = np.array(delta_data, dtype=object)

np.savez_compressed(os.path.join(dataset_path, "db_1_data.npz"), emg=emg_data, delta=delta_data)

## Next steps

We now have the EMG signals to use as input to the network, and the dependent variable for regression, the $\delta\left[n\right]$. This should suffice for the finger regression task, however, we'll
try different approaches, such as:
- Different network architectures: Vanilla RNN, LSTM, Bi-directional LSTM
- Use of covariates, such as the physiological data of the subject (sex, age, height, weight, available in the ninapro dataset DB1), to try to
improve inter-subject accuracy.
- Use of amputated patient data from the Ninapro dataset.

The next step is to break down the data into batches, and then try training a RNN with differnt input sequence lenghts.

## Batching

In [70]:
# load data
import os
from scipy.io import loadmat
import numpy as np

window_size = 200 # splits all datapoints into 200 samples

def transpose_array(array):
    for idx in range(len(array)):
        array[idx] = np.transpose(array[idx])
    array_t = array
    return array_t

# load processed file
# put samplesXchannel into channelXsamples
dataset_path = os.path.join("datasets", "ninapro", "db1")
data_load_path = os.path.join(dataset_path,"db_1_data.npz")
with np.load(data_load_path, allow_pickle=True) as data:
    emg_data = transpose_array(data['emg']) 
    delta_data = transpose_array(data['delta'])

def segment_data(emg_data, delta_data, window_size):
    inputs = []
    targets = []
    num_trials = emg_data.shape[0]
    for trial_idx in range(num_trials):
        # Calculate the largest index that is a multiple of window_size
        max_idx = (emg_data[trial_idx].shape[1] // window_size) * window_size
        #print(f'Trial {trial_idx}: data size = {emg_data[trial_idx].shape[1]}, max_idx = {max_idx}')
        for start_idx in range(0, max_idx, window_size):
            input_t = emg_data[trial_idx][:,start_idx:start_idx + window_size]
            target_t = delta_data[trial_idx][:,start_idx:start_idx + 1]
            inputs.append(input_t)
            targets.append(target_t)

    inputs = np.stack(inputs, axis=0)
    targets = np.stack(targets, axis=0)
    
    return inputs, targets

X, Y = segment_data(emg_data, delta_data, window_size)
print(X.shape)
print(Y.shape)

#dataset_path = os.path.join("datasets", "ninapro", "db1")
#filename = os.path.join(dataset_path,'segmented_data.npz')
#np.savez_compressed(filename, inputs=X, targets=Y)

(62721, 10, 200)
(62721, 22, 1)


# TODO amanhã

Tentar treinar

## Training

In [71]:
# Define model properties
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Multivariate LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [59]:
# Load data
import torch
import numpy as np
import os

dataset_path = os.path.join("datasets", "ninapro", "db1")
filename = os.path.join(dataset_path,'segmented_data.npz')

with np.load(filename) as data:
    X = data['inputs']
    Y = data['targets']

X = torch.from_numpy(X).to(torch.float32).to(device)
Y = torch.from_numpy(Y).to(torch.float32).to(device)

In [75]:
X = torch.from_numpy(X).to(torch.float32).to(device)
Y = torch.from_numpy(Y).to(torch.float32).to(device)

In [86]:
# Training loop
input_dim = 200   # number of EMG channels
hidden_dim = 128
batch_size = 256
output_dim = Y.shape[1]   # number of Delta channels
num_layers = 2

model = LSTMModel(input_size=200, hidden_size=128, num_layers=2, output_size = 22)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

train_data = torch.utils.data.TensorDataset(X,Y)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

epochs = 100

for epoch in range(epochs):
    for i, (inputs,labels) in enumerate(train_loader):
        optimizer.zero_grad()

        outputs = model(inputs)
        #print(f"Output {outputs.shape},\t Labels {labels.shape}")
        loss = criterion(outputs,labels.squeeze(dim=2))

        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/100, Loss: 4.444336809683591e-06
Epoch 2/100, Loss: 2.279440423080814e-06
Epoch 3/100, Loss: 8.677125151734799e-08
Epoch 4/100, Loss: 2.2819439493559912e-07
Epoch 5/100, Loss: 8.526488386451092e-07
Epoch 6/100, Loss: 6.93563748654924e-08
Epoch 7/100, Loss: 1.1324183191163684e-07
Epoch 8/100, Loss: 1.2727048215310788e-07
Epoch 9/100, Loss: 9.878334594759508e-07
Epoch 10/100, Loss: 5.3693632651174994e-08
Epoch 11/100, Loss: 1.002454064291669e-05
Epoch 12/100, Loss: 2.4172257084842386e-08
Epoch 13/100, Loss: 1.0848589226952754e-06
Epoch 14/100, Loss: 8.108859219646547e-07
Epoch 15/100, Loss: 2.7670806090895894e-08
Epoch 16/100, Loss: 3.112176827357871e-08
Epoch 17/100, Loss: 1.0932136973451634e-07
Epoch 18/100, Loss: 2.7543022085296798e-08
Epoch 19/100, Loss: 5.0288917918805964e-08
Epoch 20/100, Loss: 3.3074069705207876e-08
Epoch 21/100, Loss: 1.205842323770412e-07
Epoch 22/100, Loss: 2.624556714181381e-07
Epoch 23/100, Loss: 2.9529328315902603e-08
Epoch 24/100, Loss: 2.0813640730

In [95]:
X[60000].shape
out_test = model(test)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
Y.shape

In [ ]:
def has_inf_or_nan(tensor):
    has_inf = torch.isinf(tensor).any()
    has_nan = torch.isnan(tensor).any()
    return has_inf or has_nan

In [ ]:
batch_size = 4
input_batch = X[:batch_size]  # Replace 'batch_size' with the actual size you want to test
input_batch = input_batch.clone().detach().requires_grad_(True)
input_batch = torch.tensor(input_batch, dtype=torch.float)
model.eval()
with torch.no_grad():
    # Pass the input batch through the model
    test_output = model(input_batch)

# Print the output from the untrained model
print(test_output)

In [ ]:
# Define the model
model = LSTMModel(input_size=200, hidden_size=128, num_layers=2, output_size = 22)
model.to(device)


# Define the input and output tensors
input_tensor = torch.randn(1, 10, 200).to(device)
#output_tensor = torch.randn(1, 22, 1).to(device)
output_tensor = torch.randn(1, 22).to(device)

# Forward pass
output = model(input_tensor)

# Calculate loss
loss = torch.nn.functional.mse_loss(output, output_tensor)

# Backpropagation
loss.backward()

In [62]:
from torchsummary import summary
summary(model)

Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              301,056
├─Linear: 1-2                            2,838
Total params: 303,894
Trainable params: 303,894
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─LSTM: 1-1                              301,056
├─Linear: 1-2                            2,838
Total params: 303,894
Trainable params: 303,894
Non-trainable params: 0